In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Tweet Sentiment Extraction Final

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Tweet Sentiment Extraction Final


In [ ]:
!pip install transformers

In [ ]:
# https://github.com/huggingface/tokenizers
import tokenizers

class config:
  MAX_LEN = 96
  TOKENIZER_PATH = './input/roberta-tokenizer'
  TOKENIZER = tokenizers.ByteLevelBPETokenizer(
    vocab_file=f'{TOKENIZER_PATH}/vocab.json',
    merges_file=f'{TOKENIZER_PATH}/merges.txt',
    lowercase=True,
    add_prefix_space=True)

In [ ]:
tweet='The quick brown fox jumps over the lazy dog'
selected_text='jumps over the lazy dog'
sentiment='positive'
tokenizer=config.TOKENIZER
max_len=config.MAX_LEN

In [ ]:
# For Roberta, CLS = <s> and SEP = </s>
# <s>The quick brown fox jumps over the lazy dog</s></s>positive</s>'  
# Convert tweet to string just to ensure that nothing breaks in the future
# Next split on all kinds of spaces to remove superfluous spaces that we might have 
# Repeating the same with selected_text
tweet = " " + " ".join(str(tweet).split())
selected_text = " " + " ".join(str(selected_text).split())
print(tweet)
print(selected_text)

 The quick brown fox jumps over the lazy dog
 jumps over the lazy dog


In [ ]:
# Get selected_text start and end idx
len_st = len(selected_text) - 1
idx0 = None
idx1 = None

for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
    if " " + tweet[ind: ind+len_st] == selected_text:
        idx0 = ind
        idx1 = ind + len_st - 1
        break

print(f"idx0: {idx0}")
print(f"idx1: {idx1}")
print(f"len_st: {len_st}")
print(f"idxed tweet: {tweet[idx0: idx1+1]}")

idx0: 21
idx1: 43
len_st: 23
idxed tweet: jumps over the lazy dog


In [ ]:
# Assign 1 as target for each char in selected_text
char_targets = [0] * len(tweet)
if idx0 != None and idx1 != None:
    for ct in range(idx0, idx1 + 1):
        char_targets[ct] = 1

print(f"char_targets: {char_targets}")

char_targets: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# For RoBERTa, tokenizer.encode doesn't add any special tokens 
# Hence no need to remove anything from tokenized_tweet.ids & tokenized_tweet.offsets
tokenized_tweet = tokenizer.encode(tweet)
input_ids_original = tokenized_tweet.ids
tweet_offsets = tokenized_tweet.offsets

print(tweet)
print(selected_text)
print(input_ids_original)
print(f"offsets: {tweet_offsets}")

 The quick brown fox jumps over the lazy dog
 jumps over the lazy dog
[5, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335]
offsets: [(0, 4), (4, 10), (10, 16), (16, 20), (20, 26), (26, 31), (31, 35), (35, 40), (40, 44)]


In [ ]:
# Get ids within tweet of words that have target char
target_idx = []
for j, (offset1, offset2) in enumerate(tweet_offsets):
    if sum(char_targets[offset1: offset2]) > 0:
        target_idx.append(j)

targets_start = target_idx[0]
targets_end = target_idx[-1]

print(f"target_idx= {target_idx}")

print(targets_start)
print(targets_end)

target_idx= [4, 5, 6, 7, 8]
4
8


In [ ]:
# Hard-coded values
sentiment_id = {
        'positive': 1313,
        'negative': 2430,
        'neutral': 7974
    }

# id for <s>: 0
# id for </s>: 2
# Input for RoBERTa
input_ids = [0] + [sentiment_id[sentiment]] + [2] + [2] + input_ids_original + [2]
token_type_ids = [0, 0, 0, 0] + [0] * (len(input_ids_original) + 1) # RoBERTa doesn't care about token_type_ids 
mask = [1] * len(token_type_ids)
tweet_offsets = [(0, 0)] * 4 + tweet_offsets + [(0, 0)]
targets_start += 4
targets_end += 4

print(input_ids)
print(token_type_ids)
print(mask)
print(tweet_offsets)
print(targets_start)
print(targets_end)

[0, 1313, 2, 2, 5, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[(0, 0), (0, 0), (0, 0), (0, 0), (0, 4), (4, 10), (10, 16), (16, 20), (20, 26), (26, 31), (31, 35), (35, 40), (40, 44), (0, 0)]
8
12


In [ ]:
# Input padding: new mask, token type ids, tweet offsets
padding_length = max_len - len(input_ids)
if padding_length > 0:
    input_ids = input_ids + ([1] * padding_length)
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)
    tweet_offsets = tweet_offsets + ([(0, 0)] * padding_length)
    
print(input_ids)
print(token_type_ids)
print(mask)
print(tweet_offsets)

[0, 1313, 2, 2, 5, 2119, 6219, 23602, 13855, 81, 5, 22414, 2335, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(0, 0), (0, 0), (0, 0), (0, 0), (0, 4), (4, 10), (10, 16), (16, 20), (20, 26), (26, 31), (31, 35), (35, 40)